# Jupyter Notebook: Klassifikation mit einem Multi Layer Perceptron in PyTorch

In diesem Notebook werden wir ein **Multi Layer Perceptron (MLP)** mit **PyTorch** erstellen, um eine Klassifikationsaufgabe auf tabellarischen Daten durchzuführen. Wir verwenden den **Adult Income Dataset**, der demografische Informationen enthält und die Aufgabe stellt, vorherzusagen, ob eine Person ein Einkommen über oder unter 50.000 USD hat. Dies ist eine binäre Klassifikationsaufgabe.

Das Notebook ist in folgende Abschnitte unterteilt:
1. **Datenvorbereitung**: Laden, Bereinigen und Vorbereiten des Datensatzes.
2. **Erstellen von PyTorch Datasets und Dataloaders**: Umwandeln der Daten in ein für PyTorch geeignetes Format.
3. **Definition des MLP-Modells**: Erstellen eines einfachen MLP mit einer versteckten Schicht.
4. **Training des Modells**: Verwenden des Adam-Optimizers und der Cross-Entropy-Loss-Funktion.
5. **Evaluation des Modells**: Berechnen von Metriken wie Accuracy, Precision, Recall und F1-Score sowie Visualisierung der Confusion Matrix.

Jeder Abschnitt enthält Erklärungen und Code, um den Studierenden ein tiefes Verständnis der Konzepte zu vermitteln.

## 1. Datenvorbereitung

Zuerst müssen wir den Datensatz laden und vorbereiten. Der **Adult Income Dataset** enthält sowohl numerische als auch kategorische Merkmale, die wir verarbeiten müssen.

### 1.1 Datensatz laden

Der Datensatz kann von [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/adult) heruntergeladen werden. Für dieses Beispiel laden wir ihn direkt aus dem Internet.

In [ ]:
import pandas as pd

# URL des Datensatzes
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

# Spaltennamen
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
    "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
    "hours-per-week", "native-country", "income"
]

# Daten laden
data = pd.read_csv(url, header=None, names=columns, na_values=" ?", skipinitialspace=True)

# Erste Zeilen anzeigen
print(data.head())

### 1.2 Daten bereinigen

Wir entfernen Zeilen mit fehlenden Werten und kodieren kategorische Variablen in numerische Werte.

In [ ]:
# Fehlende Werte entfernen
data = data.dropna()

# Kategorische Variablen in numerische umwandeln
from sklearn.preprocessing import LabelEncoder

categorical_cols = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country", "income"]
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Zielvariable (income) extrahieren
X = data.drop("income", axis=1)
y = data["income"]

### 1.3 Daten in Trainings-, Validierungs- und Testsets aufteilen

Wir teilen den Datensatz in 70% Training, 15% Validierung und 15% Test.

In [ ]:
from sklearn.model_selection import train_test_split

# Aufteilung in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Aufteilung des Trainingsdatensatzes in Trainings- und Validierungsdaten
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1765, random_state=42)  # 0.1765 * 85% ≈ 15%

### 1.4 Daten skalieren

Da neuronale Netze empfindlich auf die Skalierung der Eingabedaten reagieren, standardisieren wir die numerischen Merkmale.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

## 2. Erstellen von PyTorch Datasets und Dataloaders

PyTorch arbeitet mit **Datasets** und **Dataloaders**, um Daten effizient zu laden und zu verarbeiten.

### 2.1 Benutzerdefiniertes Dataset erstellen

Wir erstellen ein benutzerdefiniertes Dataset, das die Daten in Tensoren umwandelt.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class AdultDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.long)  # Annahme: y ist ein pandas Series

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Datasets erstellen
train_dataset = AdultDataset(X_train, y_train)
val_dataset = AdultDataset(X_val, y_val)
test_dataset = AdultDataset(X_test, y_test)

### 2.2 Dataloaders erstellen

Dataloaders ermöglichen das Laden der Daten in Batches und das Mischen der Daten.

In [ ]:
batch_size = 64
device = "cuda" if torch.cuda.is_available() else "cpu"

# Dataloaders mit pin_memory=True, wenn CUDA verfügbar ist
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=device != "cpu")
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=device != "cpu")
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=device != "cpu")

## 3. Definition des MLP-Modells

Ein **Multi Layer Perceptron (MLP)** besteht aus einer Eingabeschicht, einer oder mehreren versteckten Schichten und einer Ausgabeschicht. Für dieses Beispiel verwenden wir ein einfaches MLP mit einer versteckten Schicht.

### 3.1 Modellarchitektur

- **Eingabeschicht**: Die Anzahl der Neuronen entspricht der Anzahl der Merkmale (Features) im Datensatz.
- **Versteckte Schicht**: Wir wählen 64 Neuronen mit ReLU-Aktivierungsfunktion.
- **Ausgabeschicht**: 2 Neuronen (für die binäre Klassifikation) mit Softmax-Aktivierungsfunktion.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Parameter
input_size = X_train.shape[1]  # Anzahl der Features
hidden_size = 128
num_classes = 2  # Binäre Klassifikation

# Modell instanziieren
model = MLP(input_size, hidden_size, num_classes)

model = model.to(device)

## 4. Training des Modells

Wir trainieren das Modell mit dem **Adam-Optimizer** und der **Cross-Entropy-Loss-Funktion**, die für Klassifikationsaufgaben geeignet ist.

### 4.1 Funktion zum Plotten der Verluste

Wir definieren eine Funktion, die die Trainings- und Validierungsverluste über die Epochen hinweg speichert und einen Plot erstellt.

In [ ]:
import matplotlib.pyplot as plt

def plot_losses(train_losses, val_losses, num_epochs):
    """
    Plottet die Trainings- und Validierungsverluste über die Epochen.

    Args:
        train_losses (list): Liste der durchschnittlichen Trainingsverluste pro Epoche
        val_losses (list): Liste der durchschnittlichen Validierungsverluste pro Epoche
        num_epochs (int): Anzahl der Epochen
    """
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', marker='o')
    plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss', marker='s')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Trainings- und Validierungsverluste über Epochen')
    plt.legend()
    plt.grid(True)
    plt.show()

### 4.2 Trainingsschleife

Wir trainieren das Modell für eine bestimmte Anzahl von Epochen und überwachen die Leistung auf dem Validierungsdatensatz.

In [ ]:
import torch.optim as optim

# Loss-Funktion und Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.007)

# Trainingsparameter
num_epochs = 22
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        # Daten auf das Gerät verschieben
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward Pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward Pass und Optimierung
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Durchschnittlichen Trainingsverlust speichern
    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)

    # Validierung
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    # Durchschnittlichen Validierungsverlust speichern
    val_loss = val_loss / len(val_loader)
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Verluste plotten
plot_losses(train_losses, val_losses, num_epochs)

## 5. Evaluation des Modells

Nach dem Training evaluieren wir das Modell auf dem Testdatensatz mit gängigen Metriken wie **Accuracy**, **Precision**, **Recall** und **F1-Score**. Zusätzlich visualisieren wir die **Confusion Matrix**.

### 5.1 Metriken berechnen

Wir verwenden die Funktionen von `sklearn` zur Berechnung der Metriken.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Modell auf Testdatensatz evaluieren
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Metriken berechnen
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

### 5.2 Confusion Matrix visualisieren

Die Confusion Matrix hilft uns, die Vorhersagen des Modells besser zu verstehen.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Confusion Matrix berechnen
cm = confusion_matrix(all_labels, all_preds)

# Plotten
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["<=50K", ">50K"], yticklabels=["<=50K", ">50K"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

## Zusammenfassung

In diesem Notebook haben wir ein Multi Layer Perceptron (MLP) mit PyTorch erstellt, um eine binäre Klassifikationsaufgabe auf dem Adult Income Dataset durchzuführen. Wir haben den Datensatz vorbereitet, PyTorch Datasets und Dataloaders erstellt, das MLP-Modell definiert und trainiert, und schließlich das Modell mit gängigen Metriken evaluiert. Die Confusion Matrix bietet eine visuelle Darstellung der Modellleistung.

Dieses Beispiel zeigt, wie Deep Learning auch auf tabellarischen Daten angewendet werden kann und dient als Einführung in die Verwendung von PyTorch für solche Aufgaben.

---

**Hinweis**: Stellen Sie sicher, dass alle erforderlichen Bibliotheken installiert sind, indem Sie `pip install pandas torch scikit-learn seaborn matplotlib` ausführen.